In [1]:
import numpy as np

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from icecream import ic

In [2]:
class FabricPhysicalConstants:
    # wind enegineering problem 6 defaults 
    rho= 2300 # kg/m^3 
    cp= 750 # J/(kg-K) 
    k=0.8 # W/(m-K)
    h_int = 4 # W/(m-K)
    h_ext = 4 # W/(m-K)
    T0 = 293.15 # K 
    Tinf_ext = 295.65 # K 
    Tinf_int = 295.65 # K 

    alpha = k/(cp*rho) # thermal diffusivity

In [3]:
class TransientWallConduction:
    def __init__(self, pc:FabricPhysicalConstants):
        self.pc = pc
        # x vals init in example => th = 0.10 m, dt = 0.010 m 
        thickness = 0.10 
        self.dx = 0.010
        self.x_vals = np.arange(start=0, stop=thickness, step=self.dx)
        self.M = len(self.x_vals)

        # time vals init 
        self.dt = 15 # seconds also from example 
        period = self.dt * 4 * 60/20 # 1 hour/ 
        self.times = np.arange(start=0, stop=period , step=self.dt)
        self.N = len(self.times)
        # TODO resolve difference between this time formulation and that of indoor air, also use time stamps...

        # initialize N*M matrix - time * x nodes 
        self.Ttx = np.zeros((self.N, self.M))
        self.Ttx[0,:] = self.pc.T0
        self.Ttx[0,0] = self.pc.Tinf_ext 
        self.Ttx[0, self.M-1] = self.pc.Tinf_int

        # time constant
        # # TODO check tau  
        self.tau = self.pc.alpha * self.dt / (self.dx**2)
        

    def calc_boundary_nodes(self, i):
        beta = lambda h: h*self.dx/self.pc.k
        eq = lambda Tself, Tnb, Tinf, h: (1 -2*self.tau - 2*self.tau*beta(h))*Tself + 2*self.tau*Tnb + 2*self.tau*beta(h)*Tinf
        
        # T0 at exterior, TM on interior 
        T0 = eq(self.Ttx[i,0], self.Ttx[i, 1], self.pc.Tinf_ext, self.pc.h_ext )
        TM = eq(self.Ttx[i,self.M-1], self.Ttx[i, self.M-2], self.pc.Tinf_int, self.pc.h_int)

        return T0, TM

    def calc_interior_nodes(self, i):
        Tint =  np.zeros(self.M)
        row = self.Ttx[i,:] 
        for m in range(self.M):
            m = m + 1 # avoid first and last nodes 
            if m < len(self.x_vals) - 1:
                Tint[m] = self.tau*(row[m - 1] + row[m + 1]) + (1 - 2*self.tau)*row[m]
        
        assert Tint[0] == 0 and Tint[self.M-1] == 0 

        return Tint 
    
    def calc_Tx_at_t(self, i):
        if i < self.N - 1:
            self.Ttx[i+1, :] = self.calc_interior_nodes(i)
            self.Ttx[i+1, 0], self.Ttx[i+1, self.M-1] = self.calc_boundary_nodes(i)

            return self.Ttx[i+1, :]

    def calc_all(self):
        for i in range(self.N):
            self.calc_Tx_at_t(i)

        return self.Ttx

## Initial Testing 

In [4]:
f1 = FabricPhysicalConstants()
f1.h_int = 8

In [5]:
t = TransientWallConduction(f1)
Txt = t.calc_all()

In [6]:
fig = go.Figure()

for time, x_at_time in enumerate(t.Ttx):
    fig.add_trace(go.Scatter(x=t.x_vals, y=x_at_time, mode='lines+markers', name=time))
fig.update_layout(xaxis_title='Distance Along Wall (X)',
                  yaxis_title='Temperature (ºC)',
                  title='Heat Conduction in Wall')


## Further testing 

### Interpolation 

In [7]:
t = TransientWallConduction(f1)

In [8]:
# determine slope for interpolation 
T0_indoor = 293.15
t.pc.Tinf_int = T0_indoor
t.pc.Tinf_ext

m = (t.pc.Tinf_int - t.pc.Tinf_ext)/(t.x_vals[-1] - t.x_vals[0])
m


-27.77777777777778

In [9]:
# create array of initial temperatures for nodes within the walls
T_inits = m*(t.x_vals) + t.pc.Tinf_ext
T_inits

array([295.65      , 295.37222222, 295.09444444, 294.81666667,
       294.53888889, 294.26111111, 293.98333333, 293.70555556,
       293.42777778, 293.15      ])

In [10]:
# interpolation 
# T0_indoor => Tinf_int, and then should change over time 
T0_indoor = 293.15
Tinf_ext = 295.65 

### Changing boundary conditions

In [11]:
# Tinf_ext => becomes an array of temperatures 
# Tinf_int => is passed in at each time step 

In [19]:
import sys
sys.path.insert(0, "../scripts")
import helpers as h
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px


In [13]:
a00, a01 = h.import_desired_data("A", "15T")
print(a00["Window Open"].unique(), a01["Window Open"].unique())
a00.head()
# a00 => experiment with only closed data (since have not implemented ventilation yet )


[1.] [0. 1.]


,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
0,2022-07-20 07:15:00,23.1806,51.421100,0.0,14.333333,30.555556,1.0
1,2022-07-20 07:30:00,23.3258,51.759333,0.0,14.333333,30.555556,1.0
2,2022-07-20 07:45:00,23.4242,51.619233,0.0,14.333333,30.555556,1.0
3,2022-07-20 08:00:00,23.5314,51.218100,0.0,14.666667,30.555556,1.0
4,2022-07-20 08:15:00,23.6314,50.950367,0.0,14.666667,30.555556,1.0


In [21]:
b00, b01 = h.import_desired_data("B", "15T")
print(b00["Window Open"].unique(), b01["Window Open"].unique())
b00.head()

[1. 0.] [0.]


,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
0,2022-07-27 09:00:00,23.828400,54.737933,0.0,17.111111,26.666667,1.0
1,2022-07-27 09:15:00,23.942133,54.292000,0.0,17.111111,26.666667,1.0
2,2022-07-27 09:30:00,24.083000,54.153200,0.0,17.111111,26.666667,1.0
3,2022-07-27 09:45:00,24.196767,53.891367,0.0,17.111111,26.666667,1.0
4,2022-07-27 10:00:00,24.307200,53.619067,0.0,18.555556,23.888889,1.0


In [23]:
assert(b01["Window Open"].unique() == [0]) # all closed 

In [24]:
a00["DateTime"].iloc[-1] - a00["DateTime"].iloc[0] # data goes for 4 days and is at 15 minute intervals, lets start by looking at only one day, and do 15 second intervals 

Timedelta('4 days 00:00:00')

In [25]:
mask = (b01['DateTime'] <= pd.Timedelta(1, "d") + b01["DateTime"].iloc[0]) 
b01_day = b01.loc[mask].reset_index(drop=True)
b01_day["DateTime"].iloc[-1]


Timestamp('2022-07-28 09:00:00')

In [30]:
assert pd.Timedelta("15s").seconds == t.dt

In [26]:
# need to resample to have 15 second intervals, or more generally, to match the dt of the computation 
b01_dt = b01_day.set_index(b01_day["DateTime"].values)
b01_dt = b01_dt.resample("15s").ffill()
b01_dt

,DateTime,Temp C,RH %,Room,Ambient Temp,Ambient RH,Window Open
2022-07-27 09:00:00,2022-07-27 09:00:00,24.1110,55.602333,1.0,17.111111,26.666667,0.0
2022-07-27 09:00:15,2022-07-27 09:00:00,24.1110,55.602333,1.0,17.111111,26.666667,0.0
2022-07-27 09:00:30,2022-07-27 09:00:00,24.1110,55.602333,1.0,17.111111,26.666667,0.0
2022-07-27 09:00:45,2022-07-27 09:00:00,24.1110,55.602333,1.0,17.111111,26.666667,0.0
2022-07-27 09:01:00,2022-07-27 09:00:00,24.1110,55.602333,1.0,17.111111,26.666667,0.0
...,...,...,...,...,...,...,...
2022-07-28 08:59:00,2022-07-28 08:45:00,24.3016,53.594300,1.0,16.000000,28.888889,0.0
2022-07-28 08:59:15,2022-07-28 08:45:00,24.3016,53.594300,1.0,16.000000,28.888889,0.0
2022-07-28 08:59:30,2022-07-28 08:45:00,24.3016,53.594300,1.0,16.000000,28.888889,0.0
2022-07-28 08:59:45,2022-07-28 08:45:00,24.3016,53.594300,1.0,16.000000,28.888889,0.0


In [33]:
b01_dt["Ambient Temp"][0:10] + 273.15

2022-07-27 09:00:00    290.261111
2022-07-27 09:00:15    290.261111
2022-07-27 09:00:30    290.261111
2022-07-27 09:00:45    290.261111
2022-07-27 09:01:00    290.261111
2022-07-27 09:01:15    290.261111
2022-07-27 09:01:30    290.261111
2022-07-27 09:01:45    290.261111
2022-07-27 09:02:00    290.261111
2022-07-27 09:02:15    290.261111
Freq: 15S, Name: Ambient Temp, dtype: float64

In [27]:
px.scatter(b01_dt, y="Ambient Temp")

#### TODO interpolate! sample hours -> 15seconds